In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# PART 1 — CONTENT



## 📌 Title
#### Modeling Volatility Clustering and Forecasting Risk in UK Equity Markets Using GARCH and Value-at-Risk
### 📘 1. Business Problem

#### Financial institutions must estimate potential daily losses under normal and stressed market conditions.
#### One common regulatory measure is Value-at-Risk (VaR).

##### However, financial returns exhibit:

* Volatility clustering

* Fat tails

* Time-varying variance

* Therefore, we model conditional volatility using GARCH(1,1) and compare different VaR methods.

In [ ]:
!pip install arch

# 📦 2. Import Libraries

In [ ]:
import matplotlib.pyplot as plt
import yfinance as yf
from scipy.stats import norm
from arch import arch_model

# 📊 3. Download UK Market Data

In [ ]:
ticker = "HSBA.L"

data = yf.download(ticker, start="2015-01-01", end="2024-12-31")
prices = data["Close"]

returns = prices.pct_change().dropna() * 100   # % returns
returns.head()

📘 Explanation 

We convert returns into percentage terms for easier interpretation and numerical stability in GARCH estimation.

# 📈 4. Stylized Facts of Financial Returns

## 4.1 Distribution

In [ ]:
returns.hist(bins=100, figsize=(10,5))
plt.title("Distribution of Daily Returns")
plt.show()

print("Mean:", returns.mean())
print("Std Dev:", returns.std())
print("Kurtosis:", returns.kurtosis())

## 4.2 Volatility Clustering

In [ ]:
rolling_vol = returns.rolling(30).std()

rolling_vol.plot(figsize=(12,5))
plt.title("30-Day Rolling Volatility")
plt.show()

# 📊 5. Test for ARCH Effects

In [ ]:
squared_returns = returns**2
pd.plotting.autocorrelation_plot(squared_returns)
plt.title("Autocorrelation of Squared Returns")
plt.show()

# 📊 6. Fit GARCH(1,1) Model

In [ ]:
model = arch_model(returns, vol='Garch', p=1, q=1)
garch_fit = model.fit(disp='off')

print(garch_fit.summary())

# 📈 7. Forecast Volatility

In [ ]:
forecast = garch_fit.forecast(horizon=10)
forecast_variance = forecast.variance.iloc[-1]

forecast_volatility = np.sqrt(forecast_variance)

forecast_volatility

# 📊 8. Value at Risk (VaR)

## 8.1 Historical VaR (95%)

In [ ]:
historical_var_95 = np.percentile(returns, 5)
historical_var_95

## 8.2 Parametric VaR

In [ ]:
mean = returns.mean()
std = returns.std()

parametric_var_95 = mean + norm.ppf(0.05) * std
parametric_var_95

## 8.3 GARCH-based VaR

In [ ]:
garch_var_95 = mean + norm.ppf(0.05) * forecast_volatility.iloc[0]
garch_var_95

# 📊 9. Backtesting VaR

In [ ]:
breaches = returns < parametric_var_95
breach_rate = breaches.mean()

print("Breach Rate:", breach_rate)

Expected breach rate ≈ 5%.

If much higher → model underestimates risk.